# PyBN basics

PyBN is constructed with the idea of ease of use, thus it is built from several compatible modules. The central one is the network module which defines some of the most popular flavor of boolean networks. Let's start by running a simple Boolean Network. The first ingridient we require is a graph. Since Boolean Networks are intrinsecally related to graphs we decided to separate the core structure from the Boolean network from its functionally. The graph must be presented as a list of adjacencies.

In [1]:
from pybn.graphs import uniform_graph

average_connectivity = 2.4
nodes = 8
graph = uniform_graph(nodes, average_connectivity)
graph

[(0, 0),
 (0, 4),
 (1, 2),
 (1, 3),
 (2, 0),
 (3, 2),
 (3, 3),
 (3, 6),
 (4, 1),
 (4, 7),
 (5, 2),
 (5, 4),
 (5, 5),
 (5, 6),
 (6, 2),
 (6, 3),
 (6, 5),
 (6, 6),
 (7, 4),
 (7, 6)]

Given a graph presented in this format it is easy to define the boolean network. Lets run it for 10 steps and have its state printed.

In [2]:
from pybn.networks import BooleanNetwork

network = BooleanNetwork(nodes, graph)
steps = 10

In [3]:
# Set a random initial state.
network.set_initial_state()
print(network.state)
# Perform several steps.
for _ in range(steps):
    network.step()
    print(network.state)

[1 1 0 0 1 1 0 1]
[0 1 1 1 0 0 1 0]
[0 1 1 0 1 0 0 0]
[0 1 1 1 1 0 1 0]
[0 1 1 0 1 0 0 0]
[0 1 1 1 1 0 1 0]
[0 1 1 0 1 0 0 0]
[0 1 1 1 1 0 1 0]
[0 1 1 0 1 0 0 0]
[0 1 1 1 1 0 1 0]
[0 1 1 0 1 0 0 0]


Now, since running networks without getting some information out of them is boring. We introduce the observers. Observers are in charge of extracting information as efficiently as possible from the evolution of the Boolean Networks. Let's calculate the entropy of the states of the previous network from 128 steps. The first thing we have to do is define and attach the observer to the network.

In [4]:
from pybn.observers import EntropyObserver

observers = [EntropyObserver(nodes=nodes)] 
network.attach_observers(observers)

Then we just simply need to run the network. Notice that since we are just running the network once all the errors are 0.

In [5]:
steps = 128

# Set a random initial state.
network.set_initial_state(observe=True)

# Perform several steps.
for _ in range(steps):
    network.step(observe=True)
    
# Get observer's summary.    
network.observers_summary()

Network entropy:	0.164 ± 0.318
Nodes entropy:
1.000 ± 0.000,	0.065 ± 0.000,	0.115 ± 0.000,	0.000 ± 0.000,	0.065 ± 0.000,	
0.000 ± 0.000,	0.000 ± 0.000,	0.065 ± 0.000,

Network complexity:	0.548 ± 0.868
Nodes entropy:
0.000 ± 0.000,	0.245 ± 0.000,	0.408 ± 0.000,	0.000 ± 0.000,	0.245 ± 0.000,	
0.000 ± 0.000,	0.000 ± 0.000,	0.245 ± 0.000,



# Multiple executions 

When running a network multiple times a few optimizations can be made. This time, let's use a Fuzzy Network to illustrate this and include another observer.

In [6]:
from pybn.networks import FuzzyBooleanNetwork
from pybn.observers import TransitionsObserver

First we create the graph.

In [7]:
average_connectivity = 1.8
nodes = 25
graph = uniform_graph(nodes, average_connectivity)

Then the Fuzzy Network.

In [8]:
base = 5
fuzzy = FuzzyBooleanNetwork(nodes, base, graph)

And we attach some observers. Note that the observers must be grouped with a list. The first small optimization we can do is to let the observer know in advance how many values runs will be averaging over.

In [9]:
runs = 8
observers = [EntropyObserver(nodes=nodes, runs=runs, base=base),
             TransitionsObserver(nodes=nodes, runs=runs)] 
fuzzy.attach_observers(observers)

In case we want to "prewarm" (transcient) the network before measuring anything, we can just "turn off" the observers and then turning them on againg afterwards. 

In [10]:
steps = 128
transcient = 128

# Perform several runs.
for _ in range(runs):
    
    # Set a random initial state.
    fuzzy.set_initial_state(observe=False)
    
    # Perform several pre-warm and unobserved steps.
    for _ in range(transcient):
        fuzzy.step(observe=False)
    
    # Pass the last state to the observers.
    fuzzy.update_observers()
    
    # Perform several observebed steps.
    for _ in range(steps):
        fuzzy.step(observe=True)
        
# Get observer's summary.
fuzzy.observers_summary()

Network entropy:	0.310 ± 0.280
Nodes entropy:
0.000 ± 0.000,	0.822 ± 0.011,	0.000 ± 0.000,	0.326 ± 0.245,	0.000 ± 0.000,	
0.419 ± 0.006,	0.426 ± 0.002,	0.426 ± 0.002,	0.000 ± 0.000,	0.633 ± 0.084,	
0.580 ± 0.019,	0.000 ± 0.000,	0.605 ± 0.105,	0.000 ± 0.000,	0.634 ± 0.036,	
0.407 ± 0.040,	0.000 ± 0.000,	0.661 ± 0.003,	0.601 ± 0.021,	0.000 ± 0.000,	
0.000 ± 0.000,	0.403 ± 0.038,	0.402 ± 0.039,	0.416 ± 0.023,	0.000 ± 0.000,	

Network complexity:	0.856 ± 0.807
Nodes entropy:
0.000 ± 0.000,	0.585 ± 0.042,	0.000 ± 0.000,	0.879 ± 0.739,	0.000 ± 0.000,	
0.974 ± 0.023,	0.978 ± 0.009,	0.978 ± 0.009,	0.000 ± 0.000,	0.929 ± 0.307,	
0.974 ± 0.073,	0.000 ± 0.000,	0.956 ± 0.376,	0.000 ± 0.000,	0.928 ± 0.140,	
0.965 ± 0.154,	0.000 ± 0.000,	0.896 ± 0.012,	0.959 ± 0.082,	0.000 ± 0.000,	
0.000 ± 0.000,	0.962 ± 0.147,	0.962 ± 0.151,	0.971 ± 0.090,	0.000 ± 0.000,	

Transitions entropy:	0.537 ± 0.450
Nodes entropy:
0.000 ± 0.000,	0.946 ± 0.013,	0.000 ± 0.000,	0.651 ± 0.200,	0.000 ± 0.000,	
0.908 ± 0.091,	0.